<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Preprocess-Data" data-toc-modified-id="Preprocess-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocess Data</a></span><ul class="toc-item"><li><span><a href="#Convert-Score-Into-Grade" data-toc-modified-id="Convert-Score-Into-Grade-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Convert Score Into Grade</a></span></li><li><span><a href="#Map-Grade-To-Number" data-toc-modified-id="Map-Grade-To-Number-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Map Grade To Number</a></span></li><li><span><a href="#Deal-With-The-NaN-Value" data-toc-modified-id="Deal-With-The-NaN-Value-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Deal With The NaN Value</a></span></li><li><span><a href="#Deal-With-Non-Numeric-Column" data-toc-modified-id="Deal-With-Non-Numeric-Column-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Deal With Non Numeric Column</a></span></li><li><span><a href="#Drop-Unnecessary-Column" data-toc-modified-id="Drop-Unnecessary-Column-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Drop Unnecessary Column</a></span></li><li><span><a href="#Split-Training-Set-And-Test-Set" data-toc-modified-id="Split-Training-Set-And-Test-Set-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Split Training Set And Test Set</a></span></li></ul></li><li><span><a href="#Create-Dataset-Class" data-toc-modified-id="Create-Dataset-Class-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Dataset Class</a></span><ul class="toc-item"><li><span><a href="#Create-Training-Set-Class" data-toc-modified-id="Create-Training-Set-Class-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create Training Set Class</a></span></li><li><span><a href="#Create-Test-Set-Class" data-toc-modified-id="Create-Test-Set-Class-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Create Test Set Class</a></span></li></ul></li><li><span><a href="#Create-Dataloader" data-toc-modified-id="Create-Dataloader-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Dataloader</a></span></li><li><span><a href="#Define-Model" data-toc-modified-id="Define-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define Model</a></span></li><li><span><a href="#Test-Performance" data-toc-modified-id="Test-Performance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Test Performance</a></span></li></ul></div>

##### Load Package

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import random

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

##### Set Up Seed

In [4]:
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministric = True

In [5]:
setup_seed(9)

# Load Data

In [6]:
df = pd.read_excel("Financial_E.xlsx")

In [7]:
df.head()

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score
0,18183,2002,AXP,2020-08-02,2002-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180989
1,18183,2003,AXP,2020-08-02,2003-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.161219
2,18183,2004,AXP,2020-08-02,2004-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180355
3,18183,2005,AXP,2020-08-02,2005-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.149305
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,0.613290


In [8]:
df.tail()

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score
696,114217052,2016,SYF,2021-05-30,2016-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.146550
697,114217052,2017,SYF,2022-04-10,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.156625
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.0,1751.0,25736.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.195295
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.0,1974.0,24825.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.139922
700,114217052,2020,SYF,2022-04-24,2020-12-31,17385.0,1167.0,16218.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,No,Reported,NaN,NaN,0.623711


In [9]:
df.shape

(701, 195)

In [10]:
df.dtypes

Organization ID                              int64
Fiscal Year                                  int64
Ticker                                      object
Feed Date                           datetime64[ns]
Fiscal Year End Date                datetime64[ns]
                                         ...      
ISO 14000 or EMS                            object
CO2 estimation method                       object
TRBC used for Median Calculation           float64
Unnamed: 193                               float64
Environmental Score                        float64
Length: 195, dtype: object

# Preprocess Data

## Convert Score Into Grade

In [11]:
def convert_score_t_grade(score):
    if 0 <= score <= 0.083333:
        return "D-"
    
    elif 0.083333 < score <= 0.166666:
        return "D"
    
    elif 0.166666 < score <= 0.250000:
        return "D+"
    
    elif 0.250000 < score <= 0.333333:
        return "C-"
    
    elif 0.333333 < score <= 0.416666:
        return "C"
    
    elif 0.416666 < score <= 0.500000:
        return "C+"
    
    elif 0.500000 < score <= 0.583333:
        return "B-"
    
    elif 0.583333 < score <= 0.666666:
        return "B+"
    
    elif 0.666666 < score <= 0.750000:
        return "B+"
    
    elif 0.750000 < score <= 0.833333:
        return "A-"
    
    elif 0.833333 < score <= 0.916666:
        return "A"
    
    elif 0.916666 < score <= 1:
        return "A+"

In [12]:
df["E_Grade"] = df["Environmental Score"].apply(convert_score_t_grade)

In [13]:
df["E_Grade"].value_counts()

D     188
A+    133
A      89
D+     88
B+     75
A-     52
C      24
B-     23
C-     16
C+     12
Name: E_Grade, dtype: int64

## Map Grade To Number

In [14]:
grade_to_number = {"A+": 0,
                   "A": 1,
                   "A-": 2,
                   "B+": 3,
                   "B": 4,
                   "B-": 5,
                   "C+": 6,
                   "C": 7,
                   "C-": 8,
                   "D+": 9,
                   "D": 10, 
                   "D-": 11}

In [15]:
df["E_num"] = df["E_Grade"].map(grade_to_number)

In [16]:
df

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score,E_Grade,E_num
0,18183,2002,AXP,2020-08-02,2002-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180989,D+,9.0
1,18183,2003,AXP,2020-08-02,2003-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.161219,D,10.0
2,18183,2004,AXP,2020-08-02,2004-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.180355,D+,9.0
3,18183,2005,AXP,2020-08-02,2005-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.149305,D,10.0
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,NaN,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,0.613290,B+,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,114217052,2016,SYF,2021-05-30,2016-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.146550,D,10.0
697,114217052,2017,SYF,2022-04-10,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No,Median,55101030.0,NaN,0.156625,D,10.0
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.00,1751.00,25736.0,NaN,NaN,...,NaN,NaN,NaN,No,Reported,NaN,NaN,0.195295,D+,9.0
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.00,1974.00,24825.0,NaN,NaN,...,NaN,NaN,NaN,No,Reported,NaN,NaN,0.139922,D,10.0


## Deal With The NaN Value

In [17]:
df = df.fillna(0)

In [18]:
df

,Organization ID,Fiscal Year,Ticker,Feed Date,Fiscal Year End Date,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,...,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,ISO 14000 or EMS,CO2 estimation method,TRBC used for Median Calculation,Unnamed: 193,Environmental Score,E_Grade,E_num
0,18183,2002,AXP,2020-08-02,2002-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.180989,D+,9.0
1,18183,2003,AXP,2020-08-02,2003-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.161219,D,10.0
2,18183,2004,AXP,2020-08-02,2004-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.180355,D+,9.0
3,18183,2005,AXP,2020-08-02,2005-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.149305,D,10.0
4,18183,2006,AXP,2020-08-02,2006-12-31,220996.55,9082.05,211914.5,0.0,0.0,...,0.0,0.0,0.0,No,0,0.0,0.0,0.613290,B+,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,114217052,2016,SYF,2021-05-30,2016-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,Median,55101030.0,0.0,0.146550,D,10.0
697,114217052,2017,SYF,2022-04-10,2017-12-31,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,No,Median,55101030.0,0.0,0.156625,D,10.0
698,114217052,2018,SYF,2021-07-11,2018-12-31,27487.00,1751.00,25736.0,0.0,0.0,...,0.0,0.0,0.0,No,Reported,0.0,0.0,0.195295,D+,9.0
699,114217052,2019,SYF,2022-04-10,2019-12-31,26799.00,1974.00,24825.0,0.0,0.0,...,0.0,0.0,0.0,No,Reported,0.0,0.0,0.139922,D,10.0


## Deal With Non Numeric Column

In [19]:
df = df.set_index(["Fiscal Year", "Ticker"])

In [20]:
df = df.select_dtypes(["int64", "float64"])

## Drop Unnecessary Column

In [21]:
df = df.drop(columns = ["Organization ID", "Environmental Score"])

In [22]:
df

,,CO2 Equivalents Emission Total,CO2 Equivalents Emission Direct,CO2 Equivalents Emission Indirect,Flaring of Natural Gas,Cement CO2 Equivalents Emission,Ozone-Depleting Substances,NOx Emissions,SOx Emissions,VOC Emissions,Waste Total,...,Land Environmental Impact Reduction,Resource Reduction Commitment (Inactive),Nuclear,Environmental Resource Impact Controversies,Recent Environmental Resource Impact Controversies,Environmental Supply Chain Monitoring,Real Estate Sustainability Certification,TRBC used for Median Calculation,Unnamed: 193,E_num
Fiscal Year,Ticker,,,,,,,,,,,,,,,,,,,,,
2002,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2003,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2004,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2005,AXP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2006,AXP,220996.55,9082.05,211914.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,SYF,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,55101030.0,0.0,10.0
2017,SYF,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,55101030.0,0.0,10.0
2018,SYF,27487.00,1751.00,25736.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,9.0


## Split Training Set And Test Set

In [23]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :],
                                                    df.iloc[:, -1],
                                                    test_size = 0.3,
                                                    random_state = 42)

print(x_train.shape)
print(x_test.shape)

(490, 188)
(211, 188)


# Create Dataset Class

## Create Training Set Class

In [24]:
class TrainingSet(Dataset):
    
    def __init__(self):
        xy = x_train.values
        
        self.x = xy[:, :-1].astype(np.float32)
        #self.x = scaler.fit_transform(self.x)
        self.y = xy[:, -1].astype(np.int64)
        
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
        
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [25]:
training_set = TrainingSet()

In [26]:
first_data_training = training_set[0]

feature, label = first_data_training
print(feature.shape)
print(label)

torch.Size([187])
tensor(1)


## Create Test Set Class

In [27]:
class TestSet(Dataset):
    
    def __init__(self):
        xy = x_test.values
        
        self.x = xy[:, : -1].astype(np.float32)
        #self.x = scaler.fit_transform(self.x)
        self.y = xy[:, -1].astype(np.int64)
        
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
        
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [28]:
test_set = TestSet()

In [29]:
first_data_test = test_set[0]

feature, label = first_data_test
print(feature.shape)
print(label)

torch.Size([187])
tensor(3)


# Create Dataloader

In [30]:
train_loader = DataLoader(dataset = training_set,
                          batch_size = 30,
                          shuffle = True)

test_loader = DataLoader(dataset = test_set,
                         batch_size = 30,
                         shuffle = True)

In [31]:
dataiter = iter(train_loader)

data = dataiter.next()

In [32]:
features, labels = data

print(features.shape)
print(labels.shape)

torch.Size([30, 187])
torch.Size([30])


# Define Model

In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        
        self.lin_start = nn.Linear(input_size, 800)
        #self.lin1 = nn.Linear(800, 400)
        self.lin_end = nn.Linear(800, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.lin_start(x)
        out = self.relu(out)
        
        #out = self.lin1(out)
        #out = self.relu(out)
        
        out = self.lin_end(out)
        
        return out

In [34]:
input_size = 187
output_size = 12

model = NeuralNetwork(input_size, output_size)

In [35]:
criterion = nn.CrossEntropyLoss()

In [36]:
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate)

In [37]:
num_epochs = 300

for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        
        y_pred = model(features)
        loss = criterion(y_pred, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1)%5 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step{i+1}/{len(train_loader)}, loss = {loss.item():.4f}')

epoch 1/300, step5/17, loss = 1343353.5000
epoch 1/300, step10/17, loss = 178131.8281
epoch 1/300, step15/17, loss = 137574.5156
epoch 2/300, step5/17, loss = 260972.7969
epoch 2/300, step10/17, loss = 99033.9844
epoch 2/300, step15/17, loss = 457164.5312
epoch 3/300, step5/17, loss = 456601.8750
epoch 3/300, step10/17, loss = 350729.0625
epoch 3/300, step15/17, loss = 39323.4961
epoch 4/300, step5/17, loss = 707786.3125
epoch 4/300, step10/17, loss = 806958.0625
epoch 4/300, step15/17, loss = 135000.8125
epoch 5/300, step5/17, loss = 113669.3359
epoch 5/300, step10/17, loss = 237834.7500
epoch 5/300, step15/17, loss = 1524666.2500
epoch 6/300, step5/17, loss = 59786.6133
epoch 6/300, step10/17, loss = 115505.3203
epoch 6/300, step15/17, loss = 110517.4609
epoch 7/300, step5/17, loss = 198355.8281
epoch 7/300, step10/17, loss = 182363.5625
epoch 7/300, step15/17, loss = 38697.0430
epoch 8/300, step5/17, loss = 111339.2344
epoch 8/300, step10/17, loss = 575143.3750
epoch 8/300, step15/1

epoch 64/300, step15/17, loss = 1843.7260
epoch 65/300, step5/17, loss = 216275.5312
epoch 65/300, step10/17, loss = 158213.9219
epoch 65/300, step15/17, loss = 52193.5156
epoch 66/300, step5/17, loss = 7332.5723
epoch 66/300, step10/17, loss = 22355.5586
epoch 66/300, step15/17, loss = 23903.1699
epoch 67/300, step5/17, loss = 25624.2930
epoch 67/300, step10/17, loss = 12212.0020
epoch 67/300, step15/17, loss = 188469.1875
epoch 68/300, step5/17, loss = 37568.6094
epoch 68/300, step10/17, loss = 1986.5868
epoch 68/300, step15/17, loss = 30498.0996
epoch 69/300, step5/17, loss = 13244.2393
epoch 69/300, step10/17, loss = 57727.0078
epoch 69/300, step15/17, loss = 20624.9746
epoch 70/300, step5/17, loss = 145714.5156
epoch 70/300, step10/17, loss = 74090.9609
epoch 70/300, step15/17, loss = 45741.0742
epoch 71/300, step5/17, loss = 26629.5078
epoch 71/300, step10/17, loss = 86421.8516
epoch 71/300, step15/17, loss = 3436.2649
epoch 72/300, step5/17, loss = 25679.3496
epoch 72/300, step1

epoch 129/300, step5/17, loss = 3957.1892
epoch 129/300, step10/17, loss = 15263.3506
epoch 129/300, step15/17, loss = 34609.8398
epoch 130/300, step5/17, loss = 2102.5149
epoch 130/300, step10/17, loss = 2616.1765
epoch 130/300, step15/17, loss = 37951.0078
epoch 131/300, step5/17, loss = 8126.4927
epoch 131/300, step10/17, loss = 3350.0066
epoch 131/300, step15/17, loss = 34617.9648
epoch 132/300, step5/17, loss = 5298.2495
epoch 132/300, step10/17, loss = 987.4139
epoch 132/300, step15/17, loss = 22717.4121
epoch 133/300, step5/17, loss = 9734.3457
epoch 133/300, step10/17, loss = 2394.9324
epoch 133/300, step15/17, loss = 7407.6675
epoch 134/300, step5/17, loss = 10484.3213
epoch 134/300, step10/17, loss = 21883.5801
epoch 134/300, step15/17, loss = 9141.6104
epoch 135/300, step5/17, loss = 4652.1387
epoch 135/300, step10/17, loss = 25063.7402
epoch 135/300, step15/17, loss = 10973.1152
epoch 136/300, step5/17, loss = 3138.8049
epoch 136/300, step10/17, loss = 5675.6753
epoch 136/3

epoch 193/300, step15/17, loss = 6310.1885
epoch 194/300, step5/17, loss = 4239.3931
epoch 194/300, step10/17, loss = 3066.3616
epoch 194/300, step15/17, loss = 1293.6351
epoch 195/300, step5/17, loss = 5226.9629
epoch 195/300, step10/17, loss = 707.7598
epoch 195/300, step15/17, loss = 2368.5125
epoch 196/300, step5/17, loss = 2272.4070
epoch 196/300, step10/17, loss = 15137.3662
epoch 196/300, step15/17, loss = 870.2010
epoch 197/300, step5/17, loss = 1271.3672
epoch 197/300, step10/17, loss = 368.2749
epoch 197/300, step15/17, loss = 11798.6338
epoch 198/300, step5/17, loss = 1031.5485
epoch 198/300, step10/17, loss = 1560.7804
epoch 198/300, step15/17, loss = 2003.0410
epoch 199/300, step5/17, loss = 362.7346
epoch 199/300, step10/17, loss = 7565.0396
epoch 199/300, step15/17, loss = 85.4731
epoch 200/300, step5/17, loss = 1193.9152
epoch 200/300, step10/17, loss = 3020.6008
epoch 200/300, step15/17, loss = 3747.0164
epoch 201/300, step5/17, loss = 668.7740
epoch 201/300, step10/17

epoch 259/300, step5/17, loss = 75.2375
epoch 259/300, step10/17, loss = 2811.4890
epoch 259/300, step15/17, loss = 47.0438
epoch 260/300, step5/17, loss = 2435.1792
epoch 260/300, step10/17, loss = 105.5867
epoch 260/300, step15/17, loss = 84.9499
epoch 261/300, step5/17, loss = 25.6477
epoch 261/300, step10/17, loss = 56.0125
epoch 261/300, step15/17, loss = 5.1245
epoch 262/300, step5/17, loss = 2938.7075
epoch 262/300, step10/17, loss = 1493.7947
epoch 262/300, step15/17, loss = 2177.0669
epoch 263/300, step5/17, loss = 5.1817
epoch 263/300, step10/17, loss = 1622.2250
epoch 263/300, step15/17, loss = 333.4300
epoch 264/300, step5/17, loss = 92.7437
epoch 264/300, step10/17, loss = 101.6362
epoch 264/300, step15/17, loss = 220.4914
epoch 265/300, step5/17, loss = 230.3476
epoch 265/300, step10/17, loss = 891.6937
epoch 265/300, step15/17, loss = 1391.6840
epoch 266/300, step5/17, loss = 625.4122
epoch 266/300, step10/17, loss = 1672.4388
epoch 266/300, step15/17, loss = 17.1151
epo

# Test Performance

In [38]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for features_t, labels_t in test_loader:
        
        y_pred_t = model(features_t)
        
        _, predictions = torch.max(y_pred_t, 1)
        n_samples += labels_t.shape[0]
        n_correct += (predictions == labels_t).sum().item()
    
    acc = 100*n_correct/n_samples
    
    print(f'accuracy = {acc}')

accuracy = 43.60189573459716
